In [2]:
import urllib2
import sqlite3
import numpy as np
import pandas as pd
from sys import stdout
from datetime import datetime
import json

foursquaredata = 0
useWholeDB = False
if useWholeDB:
    con = sqlite3.connect('data/tweetsSpring.db')
    foursquaredata = pd.read_sql("SELECT * from tweets WHERE source = 'Foursquare'", con)
else:
    con = sqlite3.connect('data/foursquareSpring.db')
    foursquaredata = pd.read_sql("SELECT * from tweets", con)  

In [2]:
from bs4 import BeautifulSoup
def extract_urls(lst):
        urls = []
        for j in lst.split(' '):
            if j.startswith('http'):
                urls.append(j)
        return urls

def resolve_url(starturl):
    try:
        req = urllib2.Request(starturl)
        res = urllib2.urlopen(req, timeout = 2)
        finalurl = res.geturl()
        soup = BeautifulSoup(res.read())
        #print soup.find_all('script')
        #print(soup.prettify())
        
        return soup
    except:
        pass

In [4]:
print len(foursquaredata)

66342


In [ ]:
for iteration in range(2,6):
    print iteration
    htmls = []
    for index, row in foursquaredata[iteration*10000:(iteration+1)*10000].iterrows():
        lst = row.content
        urls = extract_urls(lst)
        if len(urls):
            htmls.append(resolve_url(urls[0]))
        progress(index-10000,10000)
        
    scripts = []
    for html in htmls:
        if (html is not None):
            if len(html.find_all('script'))>10:
                scripts.append([str(html.find_all('script')[10])])
            else:
                scripts.append([])
        else:
            scripts.append([])
            
    with open('data/script'+str(iteration+1)+'.p', 'wb') as fileToSave:
        np.save(fileToSave, scripts)

2
100.210000%

In [ ]:
redirects[0].find_all('script')[10]

In [3]:
    def progress(i, n):
        stdout.write("\r%f%%" % (i*100/float(n)))
        stdout.flush()
        if i == n-1:
            stdout.write("\r100%")
            print("\r\n")

In [5]:
len(htmls)

10000

In [14]:
scripts = []
for html in htmls:
    if (html is not None):
        if len(html.find_all('script'))>10:
            scripts.append([str(html.find_all('script')[10])])
        else:
            scripts.append([])
    else:
        scripts.append([])

In [17]:
with open('data/script2.p', 'wb') as fileToSave:
    np.save(fileToSave, scripts)

In [22]:
with open('data/script4.p', 'r') as fileToSave:
    theScripts = np.load(fileToSave)

In [25]:
checkinStopWord = 'checkin: '
venueStopWord = ',venue: ' 
parsedCheckins = []
for scriptArray in theScripts:
    if len(scriptArray):
        scriptString = scriptArray[0]
        checkInJSON = scriptString[scriptString.find(checkinStopWord)+len(checkinStopWord):scriptString.find(venueStopWord)]
        try:
            parsedCheckins.append(json.loads(checkInJSON))
        except:
            print 'Failed to parse JSON'

Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON
Failed to parse JSON


In [11]:
json.loads(checkInJSON)

{u'canonicalPath': u'/margosanfirova/checkin/552a9b9e498ee5da5dbf273c',
 u'canonicalUrl': u'https://foursquare.com/margosanfirova/checkin/552a9b9e498ee5da5dbf273c',
 u'createdAt': 1428855710,
 u'id': u'552a9b9e498ee5da5dbf273c',
 u'likes': {u'count': 0, u'groups': []},
 u'photos': {u'count': 0, u'items': []},
 u'reasonCannotAddComments': u'loggedout',
 u'reasonCannotSeeComments': u'loggedout',
 u'score': {u'_scores': [{u'icon': u'https://ss1.4sqi.net/img/points/swarm-mayor.png',
    u'message': u'5 check-ins at Movie Theaters in the past 2 months.',
    u'points': 3}],
  u'total': 3},
 u'shout': u'\u0420\u044b\u0434\u0430\u0435\u043c \u0441 \u041e\u043b\u0435\u0439 \u043f\u043e\u0441\u043b\u0435 \u043f\u0440\u043e\u0441\u043c\u043e\u0442\u0440\u0430 \u0424\u043e\u0440\u0441\u0430\u0436 7\U0001f62d\U0001f62d',
 u'source': {u'name': u'Swarm for iOS', u'url': u'https://www.swarmapp.com'},
 u'timeZoneOffset': 240,
 u'type': u'checkin',
 u'user': {u'canonicalPath': u'/margosanfirova',
  u'c

In [24]:
with open('data/checkins.json', 'w') as outfile:
    json.dump(parsedCheckins, outfile)